In which I attempt to control multiple robots simultaneously!


In [1]:
# Foundational Sphero API, forked from Simon's Library
import sphero

In [2]:
# Tracking code
import cv2
# SpheroNav base classes
from tracker.trackingfilter import FilterSpheroBlueCover, FilterSpheroYellowCover, FilterGlow

from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker
from util import Vector2D

In [3]:
# Support notebook usage of SpheroTeam
import sys
sys.path.append('../')

In [4]:
import SpheroTeam
# If project is rearchitected, rethink how to make functions accessible
# Look into python module / api design
from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points
from SpheroTeam.navigation import calibrate_bot_direction, get_bot_position, get_team_offsets

from SpheroTeam.formations import roll_polygon, roll_push
from SpheroTeam.director import bot_go_to_point

In [5]:
# General Python Imports
import time
import os
import logging

#### SpheroTeam Setup


For more details about what each line does, refer to the previous notebook (6)


In [10]:
# This is a list of the colored "initial names" of the spheros you want to work with in this session
ACTIVEBOTS = ['RPB', 
              'RWR', 
              'ORG',
             'YYP', 
             'RYR',
             'YPR',
             'GRY']


ACTIVEBOTS = ['RPB', 
             'RYR',
             'RWR',
             'ORG']

ACTIVEBOTS = ['YPR']

              
#              'RYR',
#              'RWR',
#              'ORG']

In [11]:
from SpheroTeam import initialize

In [12]:
PROJ_ROOT = os.pardir
CONFIG = os.path.join(PROJ_ROOT, "config.json")
config = readJsonFile(CONFIG)
TEAM_COLORS = config['teamColors']

# Initialize to manage sphero objects
manager = sphero.SpheroManager()

# Initialize Sphero manager using options provided in the config file
manager= initialize.load_sphero_roster(manager, CONFIG)

# Connect to robots
bots = initialize.connect_sphero_team(manager, ACTIVEBOTS)

Sphero-YPR Try #0
Connected!


In [13]:
# Check power, robots shut down after dropping below 7 volts
SpheroTeam.print_team_status(bots)



Sphero-YPR:Battery OK | 8.2 V


In [14]:
TEAM_COLORS = config['teamColors']
TEAM_COLORS


[[255, 0, 0],
 [0, 255, 0],
 [0, 0, 255],
 [255, 0, 255],
 [255, 255, 0],
 [255, 133, 0],
 [255, 192, 203]]

In [15]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)



#### Camera Tracker Setup



In [16]:
tracker = ColorTracker(config['cameraID'])

In [17]:
imageX, imageY = tracker.image_size

print "Image: {} x {} pixels".format(imageX, imageY)

Image: 640 x 480 pixels


In [18]:
# For now, blue is the only color we can reliably track
traceable_color = [0, 0, 255]  # RGB Blue
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [19]:
# make sure everyone's light is off before calibrating.

for bot in bots:
    bot.set_rgb(0,0,0)

In [20]:
# Blue seems to be the most reliable color to track with.
# Be suspicious if any of the offsets are 0!!!
# on reflection, may want method to be able to redo 

# the offsets of just a few select robots
offsets = get_team_offsets(bots, traceable_blue, traceable_color, tracker)

Start (147,151)
End   (195,136)
Angle 342.645975364
'q' to quit, else calibrate next robot a


In [ ]:
# to redo individual offsets, work like this
def redo_offset(offsets, bots, index, traceable_object, traceable_color)
offset

In [7]:
?roll_polygon

In [21]:
# Roll in a square to show it's working
# can roll in single direction using roll-push.
roll_polygon(bots, 1, offsets, 0, speed=120, TIMEOUT=1000)


Dispatch Time 0.0169999599457


In [21]:
for i, bot in enumerate(bots):
    SpheroTeam.roll_sphero(bot, 50, 0, offsets[i])
    time.sleep(2)

In [45]:
roll_polygon(bots, 3, offsets, 0, speed=80, TIMEOUT=1500)

Dispatch Time 0.232999801636
Dispatch Time 0.24799990654
Dispatch Time 0.197999954224


### Finally: Group sending of robots to points




One problem: when robot rolls off the viewing field. Maybe when a robot cannot be seen, it should roll in the opposite of the last angle it rolled in.


In [15]:
?get_bot_position()

In [ ]:
print imageX
print imageY

In [49]:
def team_go_to_points(bots, targets, offsets, traceable_object, traceable_color,
                      MAX_X=imageX, MAX_Y=imageY, TIMEOUT=200, stopRadius=25):
    '''
        Send each bot in the team to its designated point. 
        Bots drive in a straight line to
        each point.
        
        Alternate function would send each robot along paths (sequences of points)
        Perhaps each point might have its own Kp or timeout or speed.
    '''
    
    for bot in bots:
        bot.set_rgb(0,0,0)
    
    for i, bot in enumerate(bots):
        print("Bot {}".format(bot.bt_name))
        bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
        time.sleep(2.5)
        
        targetX, targetY = targets[i]
        bot_go_to_point(bot, offsets[i], targetX, targetY,          
                        traceable_object, traceable_color, MAX_X, MAX_Y, tracker, 
                        TIMEOUT=250, stopRadius=stopRadius)  
        
        bot.set_rgb(0,0,0)
        

In [50]:
def team_go_to_paths(bots, paths, offsets, traceable_object, traceable_color,
                      MAX_X=imageX, MAX_Y=imageY, TIMEOUT=200, stopRadius=25):
    '''
        Send each bot in the team some points!
    '''
    
    for bot in bots:
        bot.set_rgb(0,0,0)
    
    for i, bot in enumerate(bots):
        print("Bot {}".format(bot.bt_name))
        bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
        time.sleep(2)
        

        for target in paths[i]:
            targetX, targetY = target
            bot_go_to_point(bot, offsets[i], targetX, targetY,          
                traceable_object, traceable_color, MAX_X, MAX_Y, tracker, TIMEOUT=250,
                           stopRadius=stopRadius)  
        
        # Turn robot off when done
        bot.set_rgb(0,0,0) 

In [51]:
# Every robot, after some pushing activity, will use this path to skirt the perimeter
recoveryPaths = [
    [
        [320, 400],  # 
        [100, 245]
    ],
    
    [
        [320, 100],
        [100, 235]
    ],
    
    [
        [320, 400],  # 
        [100, 250]
    ],
    
    [
        [320, 100],
        [100, 230]
    ],
    
    [
        [320, 400],  # 
        [100, 255]
    ],
    [
        
        [320, 100],
        [100, 225]
    ],
    [
        [320, 100],
        [100, 225]
    ]
]

In [57]:
team_go_to_points([bots[2]], [[50, 50]], [offsets[2]], traceable_blue, traceable_color, stopRadius=15, )







Bot Sphero-RWR
Go 579,434 to 50,50| Distance 653.679585118 / -144.024180558


Stopped at 33,57, with dist 18.3847763109


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

In [24]:
# Send robots to a cluster around 300, 150
targets = [ [300, 150 + i*5] for i in range(len(bots))] # space out the robots along x = 100


team_go_to_points(bots, targets, offsets, traceable_blue, traceable_color, stopRadius=15, )

Bot Sphero-RPB
Go 354,362 to 300,150| Distance 218.769284864 / -104.290325082
Stopped at 307,159, with dist 11.401754251
Bot Sphero-RWR
Go 253,308 to 300,155| Distance 160.056240116 / -72.9235897151
Stopped at 285,181, with dist 30.0166620396
Bot Sphero-ORG
Go 332,284 to 300,160| Distance 128.062484749 / -104.4702941
Stopped at 255,144, with dist 47.759815745
Bot Sphero-YYP
Go 303,76 to 300,165| Distance 89.0505474436 / 91.9305874412
Stopped at 326,157, with dist 27.2029410175
Bot Sphero-RYR
Go 297,238 to 300,170| Distance 68.066144301 / -87.4738830884
Stopped at 328,53, with dist 120.303782152
Bot Sphero-YPR
Go 437,143 to 300,175| Distance 140.687597179 / 166.852757625


Stopped at 373,388, with dist 225.162163784
Bot Sphero-GRY
Go 335,402 to 300,180| Distance 224.742074388 / -98.9593729072


Stopped at 84,259, with dist 229.993478168


### Now, move them together




In [28]:
SpheroTeam.set_team_colors(bots, TEAM_COLORS)    

In [29]:
# Test a pentagon
roll_polygon(bots, 5, offsets, 180, speed=60, TIMEOUT=1200)

Dispatch Time 0.162999868393
Dispatch Time 0.181999921799
Dispatch Time 0.164999961853
Dispatch Time 0.197000026703
Dispatch Time 0.164000034332


In [61]:
# Bring a lost sphero back if it rolled away (consider making an arena wall in future)
# SpheroTeam.roll_sphero(bots[0], 40, 180, offsets[0])

In [ ]:
# Roll all team along angle 0 at a speed of 70 for 3 seconds
roll_push(bots, 0, 70, offsets, TIMEOUT=3000)

In [30]:
# Show how robots would recover from "overshooting" the target by going to the outside of the arena
#, and then returning to a home area.
team_go_to_paths(bots, recoveryPaths, offsets, traceable_blue, traceable_color)

Bot Sphero-RPB
Get from 248,130 to 320,400| Distance 279.435144533 / 75.0685828219
Stopped at 358,408, with dist 38.8329756779
Get from 355,387 to 100,245| Distance 291.871547089 / -150.888195474
Stopped at 135,253, with dist 35.902646142
Bot Sphero-RWR
Get from 134,236 to 320,100| Distance 230.417013261 / -36.1736290458
Stopped at 319,82, with dist 18.0277563773
Get from 317,87 to 100,235| Distance 262.665186121 / 145.70493089
Stopped at 103,219, with dist 16.2788205961
Bot Sphero-ORG
Get from 90,237 to 320,400| Distance 281.902465402 / 35.3250346381
Stopped at 302,417, with dist 24.7588368063
Get from 326,432 to 100,250| Distance 290.172362571 / -141.155168343
Stopped at 92,232, with dist 19.6977156036


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 422, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 306, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

Challenges: 

- Path planning is not the same thing as simply choosing to arrive at a point.
    - Need to come up with algorithm to avoid obstacles, knowing that you may miss goal points by 10-20 centimeters.
    
- Tuning the right KP parameter is tricky
- Coming up with recovery paths is tricky

- How to stop robots from bumping into each other


- When robot runs out of bounds, we can't see it... need to create a "fence" around the arena and that would fix it.

- Tracker can only reliably "see" 1 color at once, so workaround is only using that 1 color for tracking.

Alternate ideas that weren't implemented due to lack of time

- When swarm needs to converge on 1 point: have everyone drive at fixed speed

- drive for however long that distance takes in pixels

- Let everyone drive simultaneously by giving each robot a blind directive, and not using any feedback to tune its behavior (aka a "formation")


- Write a function that given a singular (x,y) point, generates a list of pixels that each robot would aim for, along with an "shape" and relevant parameters for how the robots should be arranged (aka box, line, circle, hexagon, etc)

- Write a function that given a singular target (x,y) point and a robot, define a list of points that the robot should visit before reaching the target points

